In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score

C:\Users\marku\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Load dataset
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\titanic.csv')

# EDA in Oblig1 Classification notebook

In [3]:
# Sets null values equal to the median of that column
def handle_null_median(df):
    # Need to set inplace=True, so it doesn't create a copy of the dataframe. Tried without and this led to null-values not being removed
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna('S', inplace=True)

    return  df

df = handle_null_median(df)
df['Sex'] = df['Sex'].replace(['female', 'male'], [0,1])
df['FamilyMembersCount'] = df['Parch'] + df['SibSp'] + 1
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch', 'Embarked'], axis=1)

In [4]:
def bins(df): # This is kind of feature engineering as well
    df['AgeGroup'] = 0
    df.loc[df['Age'] < 16.336, 'AgeGroup' ] = 0
    df.loc[(df['Age'] >= 16.336) & (df['Age'] < 32.252), 'AgeGroup'] = 1
    df.loc[(df['Age'] >= 32.252) & (df['Age'] < 48.168), 'AgeGroup'] = 2
    df.loc[(df['Age'] >= 48.168) & (df['Age'] < 64.084), 'AgeGroup'] = 3
    df.loc[df['Age'] >= 64.084, 'AgeGroup'] = 4

    return df

df = bins(df)

# XGBoost

In [5]:
df_XGB = df.copy()

In [6]:
df_XGB = df_XGB.drop(['Age', 'Fare'], axis=1)

In [7]:
df_XGB['Pclass'] = df_XGB['Pclass'].astype('category')
df_XGB['AgeGroup'] = df_XGB['AgeGroup'].astype('category')

df_XGB = pd.get_dummies(df_XGB)

df_XGB.head()

,Survived,Sex,FamilyMembersCount,Pclass_1,Pclass_2,Pclass_3,AgeGroup_0,AgeGroup_1,AgeGroup_2,AgeGroup_3,AgeGroup_4
0,0,1,2,0,0,1,0,1,0,0,0
1,1,0,2,1,0,0,0,0,1,0,0
2,1,0,1,0,0,1,0,1,0,0,0
3,1,0,2,1,0,0,0,0,1,0,0
4,0,1,1,0,0,1,0,0,1,0,0


In [8]:
features = df_XGB.drop('Survived', axis=1)
targets = df_XGB['Survived']
train_X, test_X, train_y, test_y = train_test_split(features, targets, random_state=42)

XGBModel = XGBClassifier(verbosity=0)

XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8098866569408596, 0.027404354702066976)

In [9]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8161434977578476

In [10]:
(XGBModel.score(train_X, train_y), XGBModel.score(test_X, test_y))

(0.8398203592814372, 0.8161434977578476)

In [11]:
f1_score(test_y, pred)

0.7573964497041421

Here I've created a XGB model which I will use as a baseline to compare with the final tuned model

# XGBOOST TUNING: Random grid search

In [17]:
XGBParamRandom = {
    'gamma':np.arange(0.1,1,0.01),
    'learning_rate':np.arange(0.1,1,0.01),
    'max_depth':np.arange(2,15,1),
    'min_child_weight':np.arange(2,15,1),
    'n_estimators':np.arange(250,400,10),
    'subsample':np.arange(0.1,1,0.1)
}

XGB_GridRandom = RandomizedSearchCV(XGBClassifier(verbosity=0), XGBParamRandom)
XGB_GridRandom.fit(train_X, train_y)
print(XGB_GridRandom.best_params_, XGB_GridRandom.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

{'subsample': 0.9, 'n_estimators': 290, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.9699999999999995, 'gamma': 0.6099999999999998} 0.8114128605094827


In [20]:
XGBModel = XGBClassifier(verbosity=0,
                         n_estiamtors=290,
                         subsample=0.7,
                         max_depth=10,
                         min_child_weight=7,
                         gamma=0.73,
                         learning_rate=0.62)
XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8173605655930872, 0.044555126168702525)

In [21]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.820627802690583

In [22]:
(XGBModel.score(train_X, train_y), XGBModel.score(test_X, test_y))

(0.8278443113772455, 0.820627802690583)

In [23]:
f1_score(test_y, pred)

0.7590361445783134

# Random Forest Classifier

In [144]:
df_RF = df.copy()

In [145]:
df_RF = df_RF.drop(['Age', 'Fare'], axis=1)

In [146]:
df_RF['Pclass'] = df_RF['Pclass'].astype('category')
df_RF['AgeGroup'] = df_RF['AgeGroup'].astype('category')

df_RF = pd.get_dummies(df_RF)

df_RF.head()

,Survived,Sex,FamilyMembersCount,Pclass_1,Pclass_2,Pclass_3,AgeGroup_0,AgeGroup_1,AgeGroup_2,AgeGroup_3,AgeGroup_4
0,0,1,2,0,0,1,0,1,0,0,0
1,1,0,2,1,0,0,0,0,1,0,0
2,1,0,1,0,0,1,0,1,0,0,0
3,1,0,2,1,0,0,0,0,1,0,0
4,0,1,1,0,0,1,0,0,1,0,0


In [147]:
features = df_RF.drop('Survived', axis=1)
targets = df_RF['Survived']
train_X, test_X, train_y, test_y = train_test_split(features, targets, random_state=42)

RFModel = RandomForestClassifier()

RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

(0.8068903602289306, 0.02970587027795773)

In [148]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

0.8161434977578476

In [149]:
(RFModel.score(train_X, train_y), RFModel.score(test_X, test_y))

(0.8413173652694611, 0.8161434977578476)

In [150]:
f1_score(test_y, pred)

0.7573964497041421

Here I've created a Random Forest model which I will use as a baseline to compare with the final tuned model

# RF TUNING (Sequential)

In [43]:
RFParam1 = {
    "n_estimators":[300,350,375],
    "max_depth":[2,4,6],
    'max_features':[5,10,15,20,31],
    "max_samples":[0.1,0.2,0.3],
    }

RF_Grid1 = GridSearchCV(RandomForestClassifier(), RFParam1)
RF_Grid1.fit(train_X, train_y)
print(RF_Grid1.best_params_, RF_Grid1.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
405 fits failed out of a total of 675.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\marku\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\marku\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\marku\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\marku\anaconda3\lib\site-packages

{'max_depth': 4, 'max_features': 5, 'max_samples': 0.3, 'n_estimators': 375} 0.823319492761755


In [59]:
RFParam2 = {
    "n_estimators":[375,400,425],
    "max_depth":[2,3,4,5,6],
    'max_features':[3,4,5,6,7],
    "max_samples":[0.25,0.3,0.35,0.4],
    }

RF_Grid2 = GridSearchCV(RandomForestClassifier(), RFParam2)
RF_Grid2.fit(train_X, train_y)
print(RF_Grid2.best_params_, RF_Grid2.best_score_)

{'max_depth': 5, 'max_features': 5, 'max_samples': 0.2, 'n_estimators': 375} 0.8233307148468185


In [171]:
RFModel = RandomForestClassifier(max_depth=5,
                                 max_samples=0.2,
                                 max_features=5,
                                 n_estimators=375)
RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

(0.8143530467960947, 0.03126335942724405)

In [172]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

0.8295964125560538

In [173]:
(RFModel.score(train_X, train_y), RFModel.score(test_X, test_y))

(0.8278443113772455, 0.8295964125560538)

In [174]:
f1_score(test_y, pred)

0.7738095238095237

# RF TUNING (Random Search)

In [169]:
RFParamRandom = {
    'max_depth':np.arange(2,15,1),
    'min_samples_split':np.arange(1,15,1),
    'max_leaf_nodes':np.arange(1,15,1),
    'min_samples_leaf':np.arange(1,15,1),
    'n_estimators':np.arange(250,450,25),
    'max_samples':np.arange(1,15,1),
    'max_features':np.arange(1,15,1)
}

RF_GridRandom = RandomizedSearchCV(RandomForestClassifier(), RFParamRandom)
RF_GridRandom.fit(train_X, train_y)
print(RF_GridRandom.best_params_, RF_GridRandom.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\marku\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\marku\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\marku\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\marku\anaconda3\lib\site-packages\jobl

{'n_estimators': 425, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_samples': 7, 'max_leaf_nodes': 5, 'max_features': 5, 'max_depth': 3} 0.7679609471439794


In [164]:
RFModel = RandomForestClassifier(n_estimators=375,
                                 min_samples_split=8,
                                 min_samples_leaf=4,
                                 max_samples=13,
                                 max_leaf_nodes=5,
                                 max_features=7,
                                 max_depth=6)
RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

(0.7799349119066322, 0.03262080808985093)

In [165]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

0.7802690582959642

In [166]:
(RFModel.score(train_X, train_y), RFModel.score(test_X, test_y))

(0.7889221556886228, 0.7802690582959642)

In [167]:
f1_score(test_y, pred)

0.6474820143884892

# GaussianNB

In [25]:
df_GNB = df.copy()

In [26]:
df_GNB = df_GNB.drop(['Fare','Age'], axis=1)

In [27]:
df_GNB = pd.get_dummies(df_GNB)

df_GNB.head()

,Survived,Pclass,Sex,FamilyMembersCount,AgeGroup
0,0,3,1,2,1
1,1,1,0,2,2
2,1,3,0,1,1
3,1,1,0,2,2
4,0,3,1,1,2


In [28]:
features = df_GNB.drop('Survived', axis=1)
targets = df_GNB['Survived']
train_X, test_X, train_y, test_y = train_test_split(features, targets, random_state=42)

GNBModel = GaussianNB()

GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

(0.7664459656604198, 0.007925493795890956)

In [29]:
GNBModel.fit(train_X, train_y)
pred = GNBModel.predict(test_X)
accuracy_score(test_y, pred)

0.7847533632286996

In [30]:
(GNBModel.score(train_X, train_y), GNBModel.score(test_X, test_y))

(0.7859281437125748, 0.7847533632286996)

In [31]:
f1_score(test_y, pred)

0.7333333333333333

# GNB TUNING

In [54]:
GNBParam = {
    'var_smoothing': [1e-13, 1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
}

GNB_Grid = GridSearchCV(GaussianNB(), GNBParam)
GNB_Grid.fit(train_X, train_y)
print(GNB_Grid.best_params_, GNB_Grid.best_score_)

{'var_smoothing': 0.01} 0.7604421501514982


In [32]:
GNBModel = GaussianNB(var_smoothing=0.01)
GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

(0.7739198743126472, 0.009502223907490172)

In [33]:
GNBModel.fit(train_X, train_y)
pred = GNBModel.predict(test_X)
accuracy_score(test_y, pred)

0.7847533632286996

In [34]:
(GNBModel.score(train_X, train_y), GNBModel.score(test_X, test_y))

(0.7904191616766467, 0.7847533632286996)

In [35]:
f1_score(test_y, pred)

0.7333333333333333

# Support Vector Machine

In [36]:
df_SVM = df.copy()

In [37]:
df_SVM = df_SVM.drop(['Fare', 'AgeGroup'], axis=1)

In [38]:
df_SVM = pd.get_dummies(df_SVM)

df_SVM.head()

,Survived,Pclass,Sex,Age,FamilyMembersCount
0,0,3,1,22.0,2
1,1,1,0,38.0,2
2,1,3,0,26.0,1
3,1,1,0,35.0,2
4,0,3,1,35.0,1


In [39]:
features = df_SVM.drop('Survived', axis=1)
targets = df_SVM['Survived']

scaler = RobustScaler()
features = scaler.fit_transform(features)

train_X, test_X, train_y, test_y = train_test_split(features, targets, random_state=42)

In [48]:
SVMModel = SVC()

SVM_scores = cross_val_score(SVMModel, train_X, train_y)
(SVM_scores.mean(), SVM_scores.std())

(0.8263270115587475, 0.0326467100098617)

In [49]:
SVMModel.fit(train_X, train_y)
pred = SVMModel.predict(test_X)
accuracy_score(test_y, pred)

0.8251121076233184

In [50]:
(SVMModel.score(train_X, train_y), SVMModel.score(test_X, test_y))

(0.8293413173652695, 0.8251121076233184)

In [51]:
f1_score(test_y, pred)

0.7692307692307693

Here I've created a SVM model which I will use as a baseline to compare with the final tuned model

# SVM TUNING

In [13]:
SVMParam = {"C":np.arange(1,10,0.25),
            'gamma':np.arange(0,1, 0.01)}
SVM_Grid = GridSearchCV(SVC(), SVMParam)
SVM_Grid.fit(train_X, train_y)
print(SVM_Grid.best_params_, SVM_Grid.best_score_)

{'C': 2.0, 'gamma': 0.08} 0.8278307709572438


C:\Users\marku\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
500 fits failed out of a total of 20000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\marku\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\marku\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\marku\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.

In [47]:
SVMModel = SVC(C=2, gamma=0.08)

SVM_scores = cross_val_score(SVMModel, train_X, train_y)
(SVM_scores.mean(), SVM_scores.std())

(0.8263270115587475, 0.03086653621707034)

In [44]:
SVMModel.fit(train_X, train_y)
pred = SVMModel.predict(test_X)
accuracy_score(test_y, pred)

0.8251121076233184

In [45]:
(SVMModel.score(train_X, train_y), SVMModel.score(test_X, test_y))

(0.8293413173652695, 0.8251121076233184)

In [46]:
f1_score(test_y, pred)

0.7692307692307693